# Câu hỏi muốn trả lời
Liệu có thể dự đoán thời tiết của 12 tiếng sau bằng cách sử dụng thông tin thời tiết của hiện tại không?

# Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
from datetime import datetime

# Đọc dữ liệu

In [2]:
raw_weather_df = pd.read_csv('raw_weather_df.csv')
raw_weather_df = raw_weather_df.drop(columns = ['Unnamed: 0'])
raw_weather_df.head()

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather_description
0,2021-01-09 00:00:00,THU DAU MOT,24.0,1011,69,20,1.54,few clouds
1,2021-01-09 01:00:00,THU DAU MOT,25.0,1012,65,98,1.54,overcast clouds
2,2021-01-09 02:00:00,THU DAU MOT,26.0,1012,65,20,1.03,few clouds
3,2021-01-09 03:00:00,THU DAU MOT,28.0,1012,57,20,2.57,few clouds
4,2021-01-09 04:00:00,THU DAU MOT,29.0,1011,51,20,1.54,few clouds


# Khám phá dữ liệu

Ý nghĩa các trường dữ liệu, theo như link API: https://openweathermap.org/api/one-call-api#list1
- time: thời gian phép đo đạc được thực hiện, tính theo giờ
- city_name: Thành phố nơi các số đo thời thiết được ghi nhận
- temp: nhiệt độ 
- pressure: áp suất không khí
- humidity: độ ẩm
- cloud: mật độ mây
- wind_speed: tốc độ gió
- weather description: đặc tả thời tiết, chi tiết các đặc tả này có thể xem tại https://openweathermap.org/weather-conditions#Weather-Condition-Codes-2

## Xem số dòng và số cột

In [3]:
raw_weather_df.shape

(1560, 8)

## Kiểm tra xem có bị lặp dòng nào không?

In [4]:
raw_weather_df.index.duplicated().sum()

0

## Xem tổng quan thông tin các trường dữ liệu

In [5]:
raw_weather_df.describe()

,temp,pressure,humidity,clouds,wind_speed
count,1560.000000,1560.000000,1560.000000,1560.000000,1560.000000
mean,24.781160,1011.043590,67.594872,32.158974,2.270968
std,3.372387,1.617146,17.125578,24.656363,1.207018
min,17.000000,1007.000000,35.000000,0.000000,0.270000
25%,22.000000,1010.000000,54.000000,20.000000,1.540000
50%,24.000000,1011.000000,68.000000,20.000000,2.060000
75%,27.000000,1012.000000,78.000000,40.000000,3.090000
max,32.610000,1015.000000,100.000000,100.000000,8.530000


In [6]:
raw_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   time                 1560 non-null   object 
 1   city_name            1560 non-null   object 
 2   temp                 1560 non-null   float64
 3   pressure             1560 non-null   int64  
 4   humidity             1560 non-null   int64  
 5   clouds               1560 non-null   int64  
 6   wind_speed           1560 non-null   float64
 7   weather_description  1560 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 97.6+ KB


Dữ liệu được lấy bằng API nên có vẻ đầy đủ

# Tiền xử lí dữ liệu

## Tạo cột output

Vì mục tiêu của tụi mình là xem thử có thể dùng dữ liệu thời tiết hiện tại để dự đoán trước 12 tiếng không nên mình sẽ thực hiện tạo thêm một cột weather_description_12 là cột output để lưu mô tả thời tiết của 12 tiếng sau  

In [7]:
raw_weather_df.insert(len(raw_weather_df.columns), "weather_description_12", "")
raw_weather_df.head()

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather_description,weather_description_12
0,2021-01-09 00:00:00,THU DAU MOT,24.0,1011,69,20,1.54,few clouds,
1,2021-01-09 01:00:00,THU DAU MOT,25.0,1012,65,98,1.54,overcast clouds,
2,2021-01-09 02:00:00,THU DAU MOT,26.0,1012,65,20,1.03,few clouds,
3,2021-01-09 03:00:00,THU DAU MOT,28.0,1012,57,20,2.57,few clouds,
4,2021-01-09 04:00:00,THU DAU MOT,29.0,1011,51,20,1.54,few clouds,


In [8]:
for row in range(0,raw_weather_df.shape[0]):
#     raw_weather_df['weather_description_12'][row+12] = raw_weather_df['weather description'][row]
     raw_weather_df.loc[row-12,'weather_description_12'] = raw_weather_df['weather_description'][row]
raw_weather_df

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather_description,weather_description_12
0,2021-01-09 00:00:00,THU DAU MOT,24.0,1011.0,69.0,20.0,1.54,few clouds,scattered clouds
1,2021-01-09 01:00:00,THU DAU MOT,25.0,1012.0,65.0,98.0,1.54,overcast clouds,scattered clouds
2,2021-01-09 02:00:00,THU DAU MOT,26.0,1012.0,65.0,20.0,1.03,few clouds,scattered clouds
3,2021-01-09 03:00:00,THU DAU MOT,28.0,1012.0,57.0,20.0,2.57,few clouds,scattered clouds
4,2021-01-09 04:00:00,THU DAU MOT,29.0,1011.0,51.0,20.0,1.54,few clouds,scattered clouds
...,...,...,...,...,...,...,...,...,...
-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,few clouds
-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,scattered clouds
-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,scattered clouds
-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,scattered clouds


Khi làm cách này ta sẽ bị dư ra bao gồm 24 dòng, bao gồm 12 dòng rỗng và 12 dòng cuối của dataframe mà cột weather_description_12 không có dữ liệu, chỉ việc bỏ các cột này

In [9]:
raw_weather_df = raw_weather_df.dropna()
raw_weather_df = raw_weather_df[raw_weather_df['weather_description_12'] != '']

Ngoài ta ta sẽ cần loại bỏ 12 dòng cuối của mỗi thành phố vì chúng lẫn với thời tiết thành phố khác

In [10]:
object = datetime.strptime('2021-01-09 00:00:00','%Y-%m-%d %H:%M:%S')
object.year

2021

In [11]:
weather_df = raw_weather_df[raw_weather_df['time'] <= '2021-01-13 11:00:00']
weather_df

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather_description,weather_description_12
0,2021-01-09 00:00:00,THU DAU MOT,24.0,1011.0,69.0,20.0,1.54,few clouds,scattered clouds
1,2021-01-09 01:00:00,THU DAU MOT,25.0,1012.0,65.0,98.0,1.54,overcast clouds,scattered clouds
2,2021-01-09 02:00:00,THU DAU MOT,26.0,1012.0,65.0,20.0,1.03,few clouds,scattered clouds
3,2021-01-09 03:00:00,THU DAU MOT,28.0,1012.0,57.0,20.0,2.57,few clouds,scattered clouds
4,2021-01-09 04:00:00,THU DAU MOT,29.0,1011.0,51.0,20.0,1.54,few clouds,scattered clouds
...,...,...,...,...,...,...,...,...,...
1543,2021-01-13 07:00:00,CAN GIO,28.0,1011.0,47.0,40.0,3.60,scattered clouds,few clouds
1544,2021-01-13 08:00:00,CAN GIO,28.0,1010.0,47.0,40.0,3.60,scattered clouds,few clouds
1545,2021-01-13 09:00:00,CAN GIO,28.0,1010.0,47.0,40.0,3.60,scattered clouds,scattered clouds
1546,2021-01-13 10:00:00,CAN GIO,27.0,1010.0,50.0,20.0,4.12,few clouds,scattered clouds


Để chuyển đổi cột output sang dạng số, tụi mình cần xem thử cột có những giá trị nào, nhưng sẽ không xem tỉ lệ của cụ thể từng giá trị để tránh làm mất tính khách quan của tập dữ liệu

In [12]:
print(weather_df['weather_description_12'].nunique())
unique_weather = weather_df['weather_description_12'].unique()
print(unique_weather)

5
['scattered clouds' 'broken clouds' 'few clouds' 'clear sky'
 'overcast clouds']


Tụi mình sẽ thực hiện đổi sang chuỗi số tương ứng 1-5 trong bước transform, thay bằng 0 nếu có giá trị khác ngoài 5 giá trị trên

## Tách tập train, validate và test

In [13]:
Y = weather_df['weather_description_12']
X = weather_df.drop(columns = ['weather_description_12'])

In [14]:
X.shape

(1404, 8)

In [15]:
Y.shape

(1404,)

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 13)

In [17]:
X_train.shape

(1123, 8)

In [18]:
X_test.shape

(281, 8)

In [19]:
Y_train.shape

(1123,)

In [29]:
Y_train

1461          few clouds
408     scattered clouds
1355    scattered clouds
120     scattered clouds
0       scattered clouds
              ...       
962           few clouds
814           few clouds
74            few clouds
188     scattered clouds
374           few clouds
Name: weather_description_12, Length: 1123, dtype: object

In [20]:
Y_test.shape

(281,)

## Khám phá lại dữ liệu huấn luyện để tiền xử lí

### Kiểu dữ liệu

In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1123 entries, 1461 to 374
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   time                 1123 non-null   object 
 1   city_name            1123 non-null   object 
 2   temp                 1123 non-null   float64
 3   pressure             1123 non-null   float64
 4   humidity             1123 non-null   float64
 5   clouds               1123 non-null   float64
 6   wind_speed           1123 non-null   float64
 7   weather_description  1123 non-null   object 
dtypes: float64(5), object(3)
memory usage: 79.0+ KB


### Xem thử cột weather_description	 có những giá trị nào

In [22]:
print(X_train['weather_description'].nunique())
print((X_train['weather_description'].value_counts(normalize=True)*100).round(2))

5
few clouds          42.12
scattered clouds    30.19
broken clouds       11.93
clear sky           11.22
overcast clouds      4.54
Name: weather_description, dtype: float64


Có vẻ là lấy thời gian khá gần nên thời tiết chủ đạo ở miền nam trong thời gian này là có mây các dạng: ít mấy, mây rải rác, mây vỡ hoặc nhiều mây hoặc trời trong xanh

In [24]:
X_train.head()

,time,city_name,temp,pressure,humidity,clouds,wind_speed,weather_description
1461,2021-01-09 21:00:00,CAN GIO,23.0,1009.0,73.0,40.0,1.54,scattered clouds
408,2021-01-11 00:00:00,TAN AN,22.0,1011.0,78.0,20.0,2.06,few clouds
1355,2021-01-10 11:00:00,CAN THO,26.0,1010.0,83.0,75.0,2.06,broken clouds
120,2021-01-09 00:00:00,THANH PHO HO CHI MINH,24.0,1011.0,69.0,20.0,1.54,few clouds
0,2021-01-09 00:00:00,THU DAU MOT,24.0,1011.0,69.0,20.0,1.54,few clouds


## Tiếp tục tiền xử lí

Các bước tiền xử lí:
- Bỏ đi cột city name vì thời tiết của các tỉnh miền nam khá giống nhau nên giữ không có ý nghĩa
- Đổi cột input sang dạng số
- Cột thời gian mình rút gọn chỉ lấy giờ, bởi vì chúng mình dự đoán dựa vào dữ liệu của 12 giờ trước nên mình nghĩ 12 giờ trước đang là thời gian nào ngày cũng khá quan trọng
- Cột weather_description tụi mình sẽ OneHotEncoder
- Cuối cùng là chuẩn hóa sau khi các cột đã có dạng số

### Bỏ cột city name

In [31]:
class DropCol(BaseEstimator, TransformerMixin):
    def _init_(self):
        return self
    def fit(self, X, Y = None):
        self.unique_weather = Y.unique()
        return self
    def transform(self, X, Y = None):
        temp_X_df = X.copy()
        temp_X_df = temp_X_df.drop(columns = ['city_name'])
        temp_Y = Y.copy()
        # Thay bằng giá trị 0 có giá trị Y khác ngoài các giá trị Y trong unique_weather
        for i in range(len(temp_Y)):
            if temp_Y.iloc[i] not in self.unique_weather:
                temp_Y.iloc[i] = 0
        temp_Y = temp_Y.replace(self.unique_weather,range(1,6))
        return temp_X_df, temp_Y

In [32]:
transformer = DropCol()
transformer.fit(X_train,Y_train)
transformer.unique_weather

array(['few clouds', 'scattered clouds', 'clear sky', 'overcast clouds',
       'broken clouds'], dtype=object)

In [33]:
processed_X_train_df, processed_Y_train = transformer.transform(X_train,Y_train)

In [35]:
processed_X_train_df.head()

,time,temp,pressure,humidity,clouds,wind_speed,weather_description
1461,2021-01-09 21:00:00,23.0,1009.0,73.0,40.0,1.54,scattered clouds
408,2021-01-11 00:00:00,22.0,1011.0,78.0,20.0,2.06,few clouds
1355,2021-01-10 11:00:00,26.0,1010.0,83.0,75.0,2.06,broken clouds
120,2021-01-09 00:00:00,24.0,1011.0,69.0,20.0,1.54,few clouds
0,2021-01-09 00:00:00,24.0,1011.0,69.0,20.0,1.54,few clouds


In [36]:
processed_Y_train.head()

1461    1
408     2
1355    2
120     2
0       2
Name: weather_description_12, dtype: int64